In [1]:
# Use Huggingface ViLT

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import os
import sys
import json
import argparse
from torchvision import transforms
from PIL import Image
import pandas as pd
from PIL import Image
from transformers import Blip2Processor
from transformers import BlipForQuestionAnswering

In [2]:
test_data = pd.read_csv('./data_test_blip/100 Test QA.txt', sep='|', header=None)
test_data.columns = ['image_id','question type', 'question', 'answer']
test_data['image path'] = './data_test_blip/VQAMed2019_Test_Images/'+test_data['image_id']+ '.jpg'
test_data

,image_id,question type,question,answer,image path
0,synpic54082,modality,what modality is shown?,cta - ct angiography,./data_test_blip/VQAMed2019_Test_Images/synpic...
1,synpic31308,modality,what modality is shown?,an - angiogram,./data_test_blip/VQAMed2019_Test_Images/synpic...
2,synpic28538,modality,what modality is shown?,xr - plain film,./data_test_blip/VQAMed2019_Test_Images/synpic...
3,synpic22377,modality,what imaging modality is used?,an - angiogram,./data_test_blip/VQAMed2019_Test_Images/synpic...
4,synpic24501,modality,what type of imaging modality is shown?,xr - plain film,./data_test_blip/VQAMed2019_Test_Images/synpic...
...,...,...,...,...,...
95,synpic39501,abnormality,what is the primary abnormality in this image?,irregular bone edema,./data_test_blip/VQAMed2019_Test_Images/synpic...
96,synpic47961,abnormality,what abnormality is seen in the image?,medulloblastoma,./data_test_blip/VQAMed2019_Test_Images/synpic...
97,synpic47550,abnormality,what is the primary abnormality in this image?,posterior sternoclavicular dislocation,./data_test_blip/VQAMed2019_Test_Images/synpic...
98,synpic24183,abnormality,what is the primary abnormality in this image?,pseudomyxoma peritonei,./data_test_blip/VQAMed2019_Test_Images/synpic...


In [3]:
# Load the model
#config = ViltConfig.from_pretrained("vilt-Med_RAD-finetuned-vqa-KHC-4-with_valid_labels")
processor = Blip2Processor.from_pretrained("Model_2/blip-saved-model")
model = BlipForQuestionAnswering.from_pretrained("Model_2/blip-saved-model").to("cuda")

In [4]:
# create a new column for the predicted answer
test_data['predicted_answer'] = ''

for i in range(len(test_data)):
    image_path = test_data['image path'][i]
    image = Image.open(image_path)
    
    text = test_data['question'][i]
    # prepare inputs
    inputs = processor(image, text, return_tensors="pt").to("cuda")

    # forward pass
    generated_ids = model.generate(**inputs, max_length=512)
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()
    
    test_data['predicted_answer'][i] = generated_text

/tmp/ipykernel_5020/3167189844.py:16: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  test_data['predicted_answer'][i] = generated_text
/tmp/ipykernel_5020/3167189844.py:16: FutureWarning: ChainedAssignmentError: behaviour will change in panda

In [5]:
test_data

,image_id,question type,question,answer,image path,predicted_answer
0,synpic54082,modality,what modality is shown?,cta - ct angiography,./data_test_blip/VQAMed2019_Test_Images/synpic...,skull and contents
1,synpic31308,modality,what modality is shown?,an - angiogram,./data_test_blip/VQAMed2019_Test_Images/synpic...,an - angiogram
2,synpic28538,modality,what modality is shown?,xr - plain film,./data_test_blip/VQAMed2019_Test_Images/synpic...,gastrointestinal
3,synpic22377,modality,what imaging modality is used?,an - angiogram,./data_test_blip/VQAMed2019_Test_Images/synpic...,vascular and lymphatic
4,synpic24501,modality,what type of imaging modality is shown?,xr - plain film,./data_test_blip/VQAMed2019_Test_Images/synpic...,xr - plain film
...,...,...,...,...,...,...
95,synpic39501,abnormality,what is the primary abnormality in this image?,irregular bone edema,./data_test_blip/VQAMed2019_Test_Images/synpic...,mr - pdw proton density
96,synpic47961,abnormality,what abnormality is seen in the image?,medulloblastoma,./data_test_blip/VQAMed2019_Test_Images/synpic...,skull and contents
97,synpic47550,abnormality,what is the primary abnormality in this image?,posterior sternoclavicular dislocation,./data_test_blip/VQAMed2019_Test_Images/synpic...,"lung, mediastinum,"
98,synpic24183,abnormality,what is the primary abnormality in this image?,pseudomyxoma peritonei,./data_test_blip/VQAMed2019_Test_Images/synpic...,gastrointestinal


In [6]:
#remove the image path column
test_data = test_data.drop(columns=['image path'])


In [7]:
#save the results
test_data.to_csv('data_test_blip/100 Test QA Predicted.csv', index=False)